In [458]:
import numpy as np
from time import time
from Crypto.Util.number import getStrongPrime
import rsa

# 1. generating prime number p, q

In [463]:
# n을 구성할 비밀키인 큰 소수 p,q를 생성하고 이들을 곱해 n을 만든다. 
# 그후 n과 서로로인 음이아닌 정수의 개수를 의미하는 정수론적 함수 phi(n)의 값을 구함
# 암호 시스템의 안전성을 위해 p,q는 Strong Prime으로 만들어준다
p = getStrongPrime(1024)
q = getStrongPrime(1024)

n= p*q

phi_n = (p-1)*(q-1)

print(p)
print(q)

132469455517305824997085724320878327237498252996816485086592318834340118744419174835168402134025934176070454595243045185269009999647878563337940291912326669148923459066979168146310264509260136294912909939563725698416946866447126287359226460886840393575612134992948641031873340339997851227554662643181337142243
151007291088844752588278501011944167544304667992683433333724215295506127492426572609106665979649186324158661108495007558377662980870628629417072454416449153870490019573353911303830961452916616278881216681976167841652255504025995947674032494374901564744851948659370094901297032519800897099769824908006898714707


In [460]:
e  = 2**(16) +1

# 2. Generating Secret Key d

In [472]:
# 비밀키인 d를 생성하기 위한 EEA함수와 secret_key_generator 함수 정의

def EEA(a,b):
    
    r_i, r_i_new = (a, b) 

    s_i, s_i_new = (1,0)
    t_i, t_i_new = (0,1)
    
    while r_i_new !=0:
        q = r_i // r_i_new
        s_i, s_i_new = (s_i_new, s_i - q*s_i_new)
        t_i, t_i_new = (t_i_new, t_i - q*t_i_new)
        r_i, r_i_new = (r_i_new, r_i - q*r_i_new)
    print(s_i)     
    return r_i, s_i, t_i
    

def secret_key_generator(phi, e):
    
    r, s, t = EEA(phi, e)
    
    
    if r !=1:
        raise ValueError("GCD of phi and e is not 1. try another p and q")
        
        
    d = t
    # secret key는 message의 지수로 골라가야 하기 때문에 양수임
    # 따라서, s*phi(n) + d*e = 1 의 식에 좌측항의 각 항목에  
    # -phi(n)*e, + phi(n)*e를 더하고 e의 계수 d+phi(n)으로 d를 업데이트 한다.
    
    while d < 0:
        d += phi(n)
    
    return d
    



In [473]:
# 요부분에서 가끔 오류가 발생(p,q를 generate 할때에 따라 오류가 났다 안났다 함)
d= secret_key_generator(phi_n, e)
print(d)

-8320
2539512980437905322777638099897448991032677986986654962360542095582253166154994497316180421796282524433706027240163160721986838120909321469416486531984567773619903558723128063259870989166865056064580642563343061626083802546254020923848687445239827210373086675874645724956759760489095938455598291774716204860200419849965859389012344891950292210857602622249551132674230599328365387003969742748612302059077126980945240728254255460402077312192988458818209817320998876287006502373404203440126287155642361010529134551127480793467580242755009663140752102494943524051060377264240738161305281467535435127355486433628155720193


# 3. Security Check

In [466]:
## 만들어낸 암호 시스템이 안전한지 확인해보자
# 1) e는 이미 2^16 +1 로 설정 했으므로 e=3일때 발생하는 공격에서 안전
# 2) 앞에서 p,q를 정의할때 이미 strong prime 중에서 선정

# 3) d > n^0.5인지 확인
d**2> n



True

# 4. encrypt function, decrypt function

In [467]:
# 암호화 함수: 영어 문장을 받으면 이것을 ascii code 숫자로 변환후 암호화를 합니다
# e는 문제이서 2^16+1로 fix했으므로 굳이 함수에서 input으로 받지 않게 설정했습니다
def encrypt(original_message ,public_key):
    # 글자로 된 메세지를 ascii code의 연속 형태로 바꾸어줍니다.
    ascii_message = ""
    for i in original_message:
        if ord(i) <100:
            ascii_message += "0"+str(ord(i))
        else:
            ascii_message += str(ord(i))
    ascii_message = int(ascii_message)
    
    encrypted_message= ascii_message
    n= public_key
    
    # e =2^16 +1로 설정했으므로, 2진법으로 표현하면 1000000000000001이다 
    # 따라서 복호화를 간단하게 하기 위해 m의 값을 제곱하는 것을 16번 반복하고, 
    # 거기에 m을 한번 더 곱해주면 된다.
    # 또한, n법 상이므로, m^x = (ny + z) 일때, (m^x)^2 = n(n*y^2 + 2nyz) + z^2이므로
    # 각각 제곱할때마다 n보다 큰 부분을 잘라 주어도, 우리의 암호화에는 지장이 없다.
    # 따라서 m^(2^16+1) 이 매우 커져 계산이 느려지는 걸 방지하기 위해 이것을 적용해줌 
    
    for i in range(16):
        encrypted_message = (encrypted_message**2) % n
    encrypted_message = encrypted_message*ascii_message % n
    
    return encrypted_message

    

In [ ]:
## 복호화 함수
# 암호화된 메세지, 비밀키, 공개키를 받습니다.
def decrypt(encrypted_message, private_key, public_key):
    c = encrypted_message
    d = private_key
    n = public_key
    
    
    # c^d 가 법n에서 원문 메세지 m과 합동이므로, m = c^d % n 을 통해 복호화가 가능하다
    # 그러나, c^d는 지나치게 큰 수라 계산이 너무 오래 걸린다.
    # 따라서 Sliding Window 알고리즘을 사용해 연산시간을 줄이고
    # 거기에 위와같이 n법상인 것을 이용해 
    
    binary_d = bin(d)[3:]
    c_powered = c
    
    for binary in binary_d:
        if binary =="1":
            c_powered = c*(c_powered**2) % n
        else:
            c_powered = c_powered**2 %n

    ascii_message = str(c_powered) 
    
    # ascii코드 메세지를 다시 영어 문장으로 전환해 줍니다
    original_message = ""
    for i in range(0,len(ascii_message),3):
        original_message += chr(int(ascii_message[i:i+3]))
    
    
    return original_message

    

# 5. actually doing encryption and decryption

In [468]:
# 실제로 전송할 암호문 원문 메세지 입니다
# 문장을 숫자 리스트로 바꾸는 과정의 편의를 위해 메세지의 맨 앞은 m으로 시작해야 함
# (ord변환시 2자리수가 되는 a,b,c나 기호들 때문에...)
real_message = "m: I love cryptography"


In [469]:
# 구현된 암호문 입니다
c = encrypt(real_message, n)
print(c)

7871468455574321940631099561937959433031794527010381170478059599058137310099142588917447372923739726132260831462722541045023354207637202331212153936864459054896079343664216748079038391896341830005432374835208323070958338853146710071653953373417334175781625705434302487958892602320898029978772135196612405474587105406796351308267341031098893086277628077642835383837022613690169822361476441569964850155248011014913381223378102684434187683552980986502005078287068439322284061697654449245284115867511898703509877515050360975988887396252805415348738328111095348660168408466115387711276152346511464347430884309260828131682


In [470]:
# 복호화 함수를 시행
decrpyted_message = decrypt(c, d, n)

In [471]:
# 쨘! 복호화된 메세지를 확인할 수 있습니다
decrpyted_message

'm: I love cryptography'